In [20]:
# Mass evacuation gymnasium tutorial

In [21]:
# import packages
import gymnasium as gym
import numpy as np
import pandas as pd
import copy
import math
import random

from gym_mass_evacuation.mass_evacuation import MassEvacuation
from gym_mass_evacuation.mass_evacuation_policy import MassEvacuationPolicy

In [22]:
# set the environment's seed and initial state that was used in Rempel (2024)
# Set the seed that was used in Rempel (2024)
rempel_2024_seed = 20180529

# Set the initial state
rempel_2024_initial_state = {
            'm_e' : {'white' : 120, 'green' : 48, 'yellow' : 8, 'red' : 1.5},
            'm_s' : {'green' : 48, 'yellow' : 72, 'red' : 120},
            'c_h' : 10,
            'c_s' : 50,
            'delta_h' : {'white' : 1, 'green' : 1, 'yellow' : 3, 'red' : 3},
            'delta_s' : {'white' : 1, 'green' : 1, 'yellow' : 3, 'red' : 3},
            'eta_h' : 3,
            'eta_sl' : 24,
            'eta_su' : 1,
            'tau_k' : 0,
            'e_k' : 0,
            'rho_e_k' : {'white' : 0, 'green' : 475, 'yellow' : 20, 'red' : 5, 'black' : 0},
            'rho_s_k' : {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0, 'black' : 0},
            'initial_helo_arrival' : [48],
            'initial_ship_arrival' : [0]
        }

In [23]:
# create the environment and policy objects
env = MassEvacuation(initial_state = rempel_2024_initial_state, \
                     seed = rempel_2024_seed,
                     default_rng = True)

policy = MassEvacuationPolicy()

/Users/mark/Research/gym_mass_evacuation/gym_mass_evacuation/mutable_priority_queue.py:48: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.queue['tau_k'][r] -= self.queue['tau_k'][r - 1]


In [24]:
# define the default action
action = {'x_hl_k' : {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0},
          'x_sl_k' : {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0},
          'x_su_k' : {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0}
         }

numTrials = 30
objectiveValue = np.zeros(numTrials)

for t in range(numTrials):
    done = False

    print('Trial : ', t, ' of', numTrials)
    while not done:

        # The action space in this is defined by three types of actions that can be taken that depend on the event that is occuring.
        # The actions are:
        # - load the helicopter with {white, green, yellow, red}
        # - load the ship with {whtie, green, yellow, red}
        # - unload the ship with {white, green, yellow, red}

        action = {'x_hl_k' : 
                  {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0},
                  'x_sl_k' : {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0},
                  'x_su_k' : {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0}
                  }

        if (env.state['e_k']) == 1:

            # load the helicopter
            params = {}
            params['total_capacity'] = env.initial_state['c_h']
            params['individual_capacity'] = env.initial_state['delta_h']
            action['x_hl_k'] = policy.green_first_loading_policy(state = env.state, params = params)
        elif (env.state['e_k'] == 2):

            # load the ship
            params = {}
            params['total_capacity'] = env.initial_state['c_s']
            params['individual_capacity'] = env.initial_state['delta_s']
            action['x_sl_k'] = policy.green_first_loading_policy(state = env.state, params = params)
        elif (env.state['e_k'] == 3):

            # unload the ship
            action['x_su_k'] = policy.white_unloading_policy(state = env.state) 

        observation, reward, terminated, truncated, info = env.step(action)
        env.state = observation

        objectiveValue[t] += reward
        done = terminated or truncated

    options = {}
    options['single_scenario'] = True
    env.reset(options = options)

print(objectiveValue)
print('Expected number of lives saved = ', sum(objectiveValue) / numTrials)

Trial :  0  of 30


/Users/mark/Research/gym_mass_evacuation/gym_mass_evacuation/mutable_priority_queue.py:48: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.queue['tau_k'][r] -= self.queue['tau_k'][r - 1]
/Users/mark/Research/gym_mass_evacuation/gym_mass_e

Trial :  1  of 30


/Users/mark/Research/gym_mass_evacuation/gym_mass_evacuation/mutable_priority_queue.py:48: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.queue['tau_k'][r] -= self.queue['tau_k'][r - 1]
/Users/mark/Research/gym_mass_evacuation/gym_mass_e

Trial :  2  of 30


...

/Users/mark/Research/gym_mass_evacuation/gym_mass_evacuation/mutable_priority_queue.py:48: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.queue['tau_k'][r] -= self.queue['tau_k'][r - 1]
/Users/mark/Research/gym_mass_evacuation/gym_mass_e

Trial :  29  of 30


/Users/mark/Research/gym_mass_evacuation/gym_mass_evacuation/mutable_priority_queue.py:48: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.queue['tau_k'][r] -= self.queue['tau_k'][r - 1]
/Users/mark/Research/gym_mass_evacuation/gym_mass_e

[168. 167. 176. 174. 166. 170. 166. 163. 173. 169. 168. 165. 159. 168.
 169. 168. 173. 163. 162. 168. 167. 169. 171. 167. 170. 167. 166. 162.
 168. 166.]
Expected number of lives saved =  167.6


/Users/mark/Research/gym_mass_evacuation/gym_mass_evacuation/mutable_priority_queue.py:48: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.queue['tau_k'][r] -= self.queue['tau_k'][r - 1]
/Users/mark/Research/gym_mass_evacuation/gym_mass_e

In [25]:
objectiveValue

array([168., 167., 176., 174., 166., 170., 166., 163., 173., 169., 168.,
       165., 159., 168., 169., 168., 173., 163., 162., 168., 167., 169.,
       171., 167., 170., 167., 166., 162., 168., 166.])

In [26]:
objectiveValue.mean()

167.6